### 2021 Census Data (ACS 5-year)for Selected Variables - Baltimore City



In [1]:
#pip install cenpy

In [2]:
#pip install openpyxl

In [3]:
# From https://cenpy-devs.github.io/cenpy/:
# Cenpy (pronounced sen-pie) is a package that automatically discovers US Census Bureau API endpoints and exposes them to Python in a consistent fashion. 
# It also provides easy-to-use access to certain well-used data products, like the American Community Survey (ACS) and 2010 Decennial Census.


In [4]:
# From https://www.census.gov/programs-surveys/acs/guidance/comparing-acs-data.html:
# "Due to the impact of the COVID-19 pandemic, the Census Bureau changed the 2020 ACS release. 
# Instead of providing the standard 1-year data products, the Census Bureau released experimental estimates from the 1-year data. 
# Data users should not compare 2020 ACS 1-year experimental estimates with any other data.""

In [5]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from census import Census
from us import states
import censusdata 
#import censusgeocode as cg
import cenpy
#from scipy.stats import linregress

# Census & gmaps API Keys
# the latest year available is the default year and so you do not need to specify the year

from config import (api_key, gkey)
c = Census(api_key)

# Configure gmaps
#gmaps.configure(api_key=gkey)

In [6]:
import os

In [7]:
os.getcwd()

'C:\\Users\\Jenkir\\Desktop\\BDC\\Commercial_corridors\\Commercial_Corridors_Demographics'

In [8]:
pd.set_option('display.max_rows', 200)

In [9]:
pd.set_option('max_colwidth', 250)

In [10]:
# see https://mcdc.missouri.edu/applications/acs/profiles/report.php?period=5&year=2020&g=06000US2451090000|05000US24510|04000US24|01000US
# for helpful cheatsheet of subject/ variables

In [11]:
# American Community Survey 5-Year Data DETAILED Tables 
# The data are population COUNTS

# See "When to Use 1-year or 5-year estimates": 
# https://www.census.gov/programs-surveys/acs/guidance/estimates.html

# Set this to false if you're trying to do this without an internet connection
# and data which would have been fetched from an API query will be read from cached files instead
INTERNET_IS_WORKING = True

if INTERNET_IS_WORKING:
    con = cenpy.remote.APIConnection('ACSDT5Y2021')
    variables = con.variables
else:
    variables = pd.read_csv('data/ACSDT5Y2021_variables.csv',index_col='Unnamed: 0')

# No matter which product you use, a cenpy APIConnection can show you the variables 
# which it can retrieve for you, returned as a pandas DataFrame.
print(f"ACSDT5Y2021 provides {len(variables)} variables.") # how many are there?
variables.head()

ACSDT5Y2021 provides 27927 variables.


,label,concept,predicateType,group,limit,predicateOnly,hasGeoCollectionSupport,attributes,required
for,Census API FIPS 'for' clause,Census API Geography Specification,fips-for,N/A,0,True,NaN,NaN,NaN
in,Census API FIPS 'in' clause,Census API Geography Specification,fips-in,N/A,0,True,NaN,NaN,NaN
ucgid,Uniform Census Geography Identifier clause,Census API Geography Specification,ucgid,N/A,0,True,True,NaN,NaN
B24022_060E,Estimate!!Total:!!Female:!!Service occupations:!!Food preparation and serving related occupations,"SEX BY OCCUPATION AND MEDIAN EARNINGS IN THE PAST 12 MONTHS (IN 2021 INFLATION-ADJUSTED DOLLARS) FOR THE FULL-TIME, YEAR-ROUND CIVILIAN EMPLOYED POPULATION 16 YEARS AND OVER",int,B24022,0,NaN,NaN,"B24022_060EA,B24022_060M,B24022_060MA",NaN
B19001B_014E,"Estimate!!Total:!!$100,000 to $124,999",HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2021 INFLATION-ADJUSTED DOLLARS) (BLACK OR AFRICAN AMERICAN ALONE HOUSEHOLDER),int,B19001B,0,NaN,NaN,"B19001B_014EA,B19001B_014M,B19001B_014MA",NaN


In [12]:
# Comments and code in this block are from https://github.com/censusreporter/nicar20-advanced-census-python/blob/master/workshop.ipynb:
# I'll use "Nicar20" as citation for the above site from here on out
# Values for 'group' are ACS table IDs; 
# For this data, when it's N/A, it's for other kinds of API variables so won't include those:
short_vars = variables[~(variables['group'] == 'N/A')] 

# Get a list of table IDs and their titles
short_vars[['group', 'concept']].drop_duplicates().sort_values('group').head(20) 

,group,concept
B01001_012E,B01001,SEX BY AGE
B01001A_002E,B01001A,SEX BY AGE (WHITE ALONE)
B01001B_029E,B01001B,SEX BY AGE (BLACK OR AFRICAN AMERICAN ALONE)
B01001C_008E,B01001C,SEX BY AGE (AMERICAN INDIAN AND ALASKA NATIVE ALONE)
B01001D_008E,B01001D,SEX BY AGE (ASIAN ALONE)
B01001E_013E,B01001E,SEX BY AGE (NATIVE HAWAIIAN AND OTHER PACIFIC ISLANDER ALONE)
B01001F_001E,B01001F,SEX BY AGE (SOME OTHER RACE ALONE)
B01001G_022E,B01001G,SEX BY AGE (TWO OR MORE RACES)
B01001H_001E,B01001H,"SEX BY AGE (WHITE ALONE, NOT HISPANIC OR LATINO)"
B01001I_027E,B01001I,SEX BY AGE (HISPANIC OR LATINO)


In [13]:
# (From Nicar20)
# Use when you know which group but still need specific API variable codes
# "attributes" column shows related variables you can request. The one that ends with M is the margin of error, and since we want to be responsible when we aggregate data, we'll be sure to aggregate the error as well. 
# Just fyi, The other two which end with A are "annotations." 
short_vars[short_vars['group'] == 'B01001'][['label','attributes']].sort_index() 

,label,attributes
B01001_001E,Estimate!!Total:,"B01001_001EA,B01001_001M,B01001_001MA"
B01001_002E,Estimate!!Total:!!Male:,"B01001_002EA,B01001_002M,B01001_002MA"
B01001_003E,Estimate!!Total:!!Male:!!Under 5 years,"B01001_003EA,B01001_003M,B01001_003MA"
B01001_004E,Estimate!!Total:!!Male:!!5 to 9 years,"B01001_004EA,B01001_004M,B01001_004MA"
B01001_005E,Estimate!!Total:!!Male:!!10 to 14 years,"B01001_005EA,B01001_005M,B01001_005MA"
B01001_006E,Estimate!!Total:!!Male:!!15 to 17 years,"B01001_006EA,B01001_006M,B01001_006MA"
B01001_007E,Estimate!!Total:!!Male:!!18 and 19 years,"B01001_007EA,B01001_007M,B01001_007MA"
B01001_008E,Estimate!!Total:!!Male:!!20 years,"B01001_008EA,B01001_008M,B01001_008MA"
B01001_009E,Estimate!!Total:!!Male:!!21 years,"B01001_009EA,B01001_009M,B01001_009MA"
B01001_010E,Estimate!!Total:!!Male:!!22 to 24 years,"B01001_010EA,B01001_010M,B01001_010MA"


In [14]:
short_vars[short_vars['group'] == 'B01003'][['label','attributes']].sort_index() 

,label,attributes
B01003_001E,Estimate!!Total,"B01003_001EA,B01003_001M,B01003_001MA"


In [15]:
short_vars[short_vars['group'] == 'B05002'][['label','attributes']].sort_index() 

,label,attributes
B05002_001E,Estimate!!Total:,"B05002_001EA,B05002_001M,B05002_001MA"
B05002_002E,Estimate!!Total:!!Native:,"B05002_002EA,B05002_002M,B05002_002MA"
B05002_003E,Estimate!!Total:!!Native:!!Born in state of residence,"B05002_003EA,B05002_003M,B05002_003MA"
B05002_004E,Estimate!!Total:!!Native:!!Born in other state in the United States:,"B05002_004EA,B05002_004M,B05002_004MA"
B05002_005E,Estimate!!Total:!!Native:!!Born in other state in the United States:!!Northeast,"B05002_005EA,B05002_005M,B05002_005MA"
B05002_006E,Estimate!!Total:!!Native:!!Born in other state in the United States:!!Midwest,"B05002_006EA,B05002_006M,B05002_006MA"
B05002_007E,Estimate!!Total:!!Native:!!Born in other state in the United States:!!South,"B05002_007EA,B05002_007M,B05002_007MA"
B05002_008E,Estimate!!Total:!!Native:!!Born in other state in the United States:!!West,"B05002_008EA,B05002_008M,B05002_008MA"
B05002_009E,Estimate!!Total:!!Native:!!Born outside the United States:,"B05002_009EA,B05002_009M,B05002_009MA"
B05002_010E,Estimate!!Total:!!Native:!!Born outside the United States:!!Puerto Rico,"B05002_010EA,B05002_010M,B05002_010MA"


In [16]:
short_vars[short_vars['group'] == 'B07007PR'][['label','attributes']].sort_index() 

,label,attributes
B07007PR_001E,Estimate!!Total:,"B07007PR_001EA,B07007PR_001M,B07007PR_001MA"
B07007PR_002E,Estimate!!Total:!!Native,"B07007PR_002EA,B07007PR_002M,B07007PR_002MA"
B07007PR_003E,Estimate!!Total:!!Foreign born:,"B07007PR_003EA,B07007PR_003M,B07007PR_003MA"
B07007PR_004E,Estimate!!Total:!!Foreign born:!!Naturalized U.S. citizen,"B07007PR_004EA,B07007PR_004M,B07007PR_004MA"
B07007PR_005E,Estimate!!Total:!!Foreign born:!!Not a U.S. citizen,"B07007PR_005EA,B07007PR_005M,B07007PR_005MA"
B07007PR_006E,Estimate!!Total:!!Same house 1 year ago:,"B07007PR_006EA,B07007PR_006M,B07007PR_006MA"
B07007PR_007E,Estimate!!Total:!!Same house 1 year ago:!!Native,"B07007PR_007EA,B07007PR_007M,B07007PR_007MA"
B07007PR_008E,Estimate!!Total:!!Same house 1 year ago:!!Foreign born:,"B07007PR_008EA,B07007PR_008M,B07007PR_008MA"
B07007PR_009E,Estimate!!Total:!!Same house 1 year ago:!!Foreign born:!!Naturalized U.S. citizen,"B07007PR_009EA,B07007PR_009M,B07007PR_009MA"
B07007PR_010E,Estimate!!Total:!!Same house 1 year ago:!!Foreign born:!!Not a U.S. citizen,"B07007PR_010EA,B07007PR_010M,B07007PR_010MA"


In [17]:
short_vars[short_vars['group'] == 'B08301'][['label','attributes']].sort_index() 

,label,attributes
B08301_001E,Estimate!!Total:,"B08301_001EA,B08301_001M,B08301_001MA"
B08301_002E,"Estimate!!Total:!!Car, truck, or van:","B08301_002EA,B08301_002M,B08301_002MA"
B08301_003E,"Estimate!!Total:!!Car, truck, or van:!!Drove alone","B08301_003EA,B08301_003M,B08301_003MA"
B08301_004E,"Estimate!!Total:!!Car, truck, or van:!!Carpooled:","B08301_004EA,B08301_004M,B08301_004MA"
B08301_005E,"Estimate!!Total:!!Car, truck, or van:!!Carpooled:!!In 2-person carpool","B08301_005EA,B08301_005M,B08301_005MA"
B08301_006E,"Estimate!!Total:!!Car, truck, or van:!!Carpooled:!!In 3-person carpool","B08301_006EA,B08301_006M,B08301_006MA"
B08301_007E,"Estimate!!Total:!!Car, truck, or van:!!Carpooled:!!In 4-person carpool","B08301_007EA,B08301_007M,B08301_007MA"
B08301_008E,"Estimate!!Total:!!Car, truck, or van:!!Carpooled:!!In 5- or 6-person carpool","B08301_008EA,B08301_008M,B08301_008MA"
B08301_009E,"Estimate!!Total:!!Car, truck, or van:!!Carpooled:!!In 7-or-more-person carpool","B08301_009EA,B08301_009M,B08301_009MA"
B08301_010E,Estimate!!Total:!!Public transportation (excluding taxicab):,"B08301_010EA,B08301_010M,B08301_010MA"


In [18]:
short_vars[short_vars['group'] == 'B11002'][['label','attributes']].sort_index() 

,label,attributes
B11002_001E,Estimate!!Total:,"B11002_001EA,B11002_001M,B11002_001MA"
B11002_002E,Estimate!!Total:!!In family households:,"B11002_002EA,B11002_002M,B11002_002MA"
B11002_003E,Estimate!!Total:!!In family households:!!In married-couple family:,"B11002_003EA,B11002_003M,B11002_003MA"
B11002_004E,Estimate!!Total:!!In family households:!!In married-couple family:!!Relatives,"B11002_004EA,B11002_004M,B11002_004MA"
B11002_005E,Estimate!!Total:!!In family households:!!In married-couple family:!!Nonrelatives,"B11002_005EA,B11002_005M,B11002_005MA"
B11002_006E,"Estimate!!Total:!!In family households:!!In male householder, no spouse present, family:","B11002_006EA,B11002_006M,B11002_006MA"
B11002_007E,"Estimate!!Total:!!In family households:!!In male householder, no spouse present, family:!!Relatives","B11002_007EA,B11002_007M,B11002_007MA"
B11002_008E,"Estimate!!Total:!!In family households:!!In male householder, no spouse present, family:!!Nonrelatives","B11002_008EA,B11002_008M,B11002_008MA"
B11002_009E,"Estimate!!Total:!!In family households:!!In female householder, no spouse present, family:","B11002_009EA,B11002_009M,B11002_009MA"
B11002_010E,"Estimate!!Total:!!In family households:!!In female householder, no spouse present, family:!!Relatives","B11002_010EA,B11002_010M,B11002_010MA"


In [19]:
short_vars[short_vars['group'] == '17001'][['label','attributes']].sort_index() 

,label,attributes


In [20]:
# poverty 
# see poverty thresholds for census at https://www.census.gov/data/tables/time-series/demo/income-poverty/historical-poverty-thresholds.html

In [21]:
short_vars[short_vars['group'] == 'C17002'][['label','attributes']].sort_index() 

,label,attributes
C17002_001E,Estimate!!Total:,"C17002_001EA,C17002_001M,C17002_001MA"
C17002_002E,Estimate!!Total:!!Under .50,"C17002_002EA,C17002_002M,C17002_002MA"
C17002_003E,Estimate!!Total:!!.50 to .99,"C17002_003EA,C17002_003M,C17002_003MA"
C17002_004E,Estimate!!Total:!!1.00 to 1.24,"C17002_004EA,C17002_004M,C17002_004MA"
C17002_005E,Estimate!!Total:!!1.25 to 1.49,"C17002_005EA,C17002_005M,C17002_005MA"
C17002_006E,Estimate!!Total:!!1.50 to 1.84,"C17002_006EA,C17002_006M,C17002_006MA"
C17002_007E,Estimate!!Total:!!1.85 to 1.99,"C17002_007EA,C17002_007M,C17002_007MA"
C17002_008E,Estimate!!Total:!!2.00 and over,"C17002_008EA,C17002_008M,C17002_008MA"


In [22]:
short_vars[short_vars['group'] == 'B15003'][['label','attributes']].sort_index() 

,label,attributes
B15003_001E,Estimate!!Total:,"B15003_001EA,B15003_001M,B15003_001MA"
B15003_002E,Estimate!!Total:!!No schooling completed,"B15003_002EA,B15003_002M,B15003_002MA"
B15003_003E,Estimate!!Total:!!Nursery school,"B15003_003EA,B15003_003M,B15003_003MA"
B15003_004E,Estimate!!Total:!!Kindergarten,"B15003_004EA,B15003_004M,B15003_004MA"
B15003_005E,Estimate!!Total:!!1st grade,"B15003_005EA,B15003_005M,B15003_005MA"
B15003_006E,Estimate!!Total:!!2nd grade,"B15003_006EA,B15003_006M,B15003_006MA"
B15003_007E,Estimate!!Total:!!3rd grade,"B15003_007EA,B15003_007M,B15003_007MA"
B15003_008E,Estimate!!Total:!!4th grade,"B15003_008EA,B15003_008M,B15003_008MA"
B15003_009E,Estimate!!Total:!!5th grade,"B15003_009EA,B15003_009M,B15003_009MA"
B15003_010E,Estimate!!Total:!!6th grade,"B15003_010EA,B15003_010M,B15003_010MA"


In [23]:
short_vars[short_vars['group'] == 'B07009'][['label','attributes']].sort_index() 

,label,attributes
B07009_001E,Estimate!!Total:,"B07009_001EA,B07009_001M,B07009_001MA"
B07009_002E,Estimate!!Total:!!Less than high school graduate,"B07009_002EA,B07009_002M,B07009_002MA"
B07009_003E,Estimate!!Total:!!High school graduate (includes equivalency),"B07009_003EA,B07009_003M,B07009_003MA"
B07009_004E,Estimate!!Total:!!Some college or associate's degree,"B07009_004EA,B07009_004M,B07009_004MA"
B07009_005E,Estimate!!Total:!!Bachelor's degree,"B07009_005EA,B07009_005M,B07009_005MA"
B07009_006E,Estimate!!Total:!!Graduate or professional degree,"B07009_006EA,B07009_006M,B07009_006MA"
B07009_007E,Estimate!!Total:!!Same house 1 year ago:,"B07009_007EA,B07009_007M,B07009_007MA"
B07009_008E,Estimate!!Total:!!Same house 1 year ago:!!Less than high school graduate,"B07009_008EA,B07009_008M,B07009_008MA"
B07009_009E,Estimate!!Total:!!Same house 1 year ago:!!High school graduate (includes equivalency),"B07009_009EA,B07009_009M,B07009_009MA"
B07009_010E,Estimate!!Total:!!Same house 1 year ago:!!Some college or associate's degree,"B07009_010EA,B07009_010M,B07009_010MA"


In [24]:
short_vars[short_vars['group'] == 'B24080'][['label','attributes']].sort_index() 

,label,attributes
B24080_001E,Estimate!!Total:,"B24080_001EA,B24080_001M,B24080_001MA"
B24080_002E,Estimate!!Total:!!Male:,"B24080_002EA,B24080_002M,B24080_002MA"
B24080_003E,Estimate!!Total:!!Male:!!Private for-profit wage and salary workers:,"B24080_003EA,B24080_003M,B24080_003MA"
B24080_004E,Estimate!!Total:!!Male:!!Private for-profit wage and salary workers:!!Employee of private company workers,"B24080_004EA,B24080_004M,B24080_004MA"
B24080_005E,Estimate!!Total:!!Male:!!Private for-profit wage and salary workers:!!Self-employed in own incorporated business workers,"B24080_005EA,B24080_005M,B24080_005MA"
B24080_006E,Estimate!!Total:!!Male:!!Private not-for-profit wage and salary workers,"B24080_006EA,B24080_006M,B24080_006MA"
B24080_007E,Estimate!!Total:!!Male:!!Local government workers,"B24080_007EA,B24080_007M,B24080_007MA"
B24080_008E,Estimate!!Total:!!Male:!!State government workers,"B24080_008EA,B24080_008M,B24080_008MA"
B24080_009E,Estimate!!Total:!!Male:!!Federal government workers,"B24080_009EA,B24080_009M,B24080_009MA"
B24080_010E,Estimate!!Total:!!Male:!!Self-employed in own not incorporated business workers,"B24080_010EA,B24080_010M,B24080_010MA"


In [25]:
short_vars[short_vars['group'] == 'B23025'][['label','attributes']].sort_index() 

,label,attributes
B23025_001E,Estimate!!Total:,"B23025_001EA,B23025_001M,B23025_001MA"
B23025_002E,Estimate!!Total:!!In labor force:,"B23025_002EA,B23025_002M,B23025_002MA"
B23025_003E,Estimate!!Total:!!In labor force:!!Civilian labor force:,"B23025_003EA,B23025_003M,B23025_003MA"
B23025_004E,Estimate!!Total:!!In labor force:!!Civilian labor force:!!Employed,"B23025_004EA,B23025_004M,B23025_004MA"
B23025_005E,Estimate!!Total:!!In labor force:!!Civilian labor force:!!Unemployed,"B23025_005EA,B23025_005M,B23025_005MA"
B23025_006E,Estimate!!Total:!!In labor force:!!Armed Forces,"B23025_006EA,B23025_006M,B23025_006MA"
B23025_007E,Estimate!!Total:!!Not in labor force,"B23025_007EA,B23025_007M,B23025_007MA"


In [26]:
short_vars[short_vars['group'] == 'B25003'][['label','attributes']].sort_index() 

,label,attributes
B25003_001E,Estimate!!Total:,"B25003_001EA,B25003_001M,B25003_001MA"
B25003_002E,Estimate!!Total:!!Owner occupied,"B25003_002EA,B25003_002M,B25003_002MA"
B25003_003E,Estimate!!Total:!!Renter occupied,"B25003_003EA,B25003_003M,B25003_003MA"


In [27]:
short_vars[short_vars['group'] == 'B25044'][['label','attributes']].sort_index() 

,label,attributes
B25044_001E,Estimate!!Total:,"B25044_001EA,B25044_001M,B25044_001MA"
B25044_002E,Estimate!!Total:!!Owner occupied:,"B25044_002EA,B25044_002M,B25044_002MA"
B25044_003E,Estimate!!Total:!!Owner occupied:!!No vehicle available,"B25044_003EA,B25044_003M,B25044_003MA"
B25044_004E,Estimate!!Total:!!Owner occupied:!!1 vehicle available,"B25044_004EA,B25044_004M,B25044_004MA"
B25044_005E,Estimate!!Total:!!Owner occupied:!!2 vehicles available,"B25044_005EA,B25044_005M,B25044_005MA"
B25044_006E,Estimate!!Total:!!Owner occupied:!!3 vehicles available,"B25044_006EA,B25044_006M,B25044_006MA"
B25044_007E,Estimate!!Total:!!Owner occupied:!!4 vehicles available,"B25044_007EA,B25044_007M,B25044_007MA"
B25044_008E,Estimate!!Total:!!Owner occupied:!!5 or more vehicles available,"B25044_008EA,B25044_008M,B25044_008MA"
B25044_009E,Estimate!!Total:!!Renter occupied:,"B25044_009EA,B25044_009M,B25044_009MA"
B25044_010E,Estimate!!Total:!!Renter occupied:!!No vehicle available,"B25044_010EA,B25044_010M,B25044_010MA"


In [28]:
short_vars[short_vars['group'] == 'B28003'][['label','attributes']].sort_index() 

,label,attributes
B28003_001E,Estimate!!Total:,"B28003_001EA,B28003_001M,B28003_001MA"
B28003_002E,Estimate!!Total:!!Has a computer:,"B28003_002EA,B28003_002M,B28003_002MA"
B28003_003E,Estimate!!Total:!!Has a computer:!!With dial-up Internet subscription alone,"B28003_003EA,B28003_003M,B28003_003MA"
B28003_004E,Estimate!!Total:!!Has a computer:!!With a broadband Internet subscription,"B28003_004EA,B28003_004M,B28003_004MA"
B28003_005E,Estimate!!Total:!!Has a computer:!!Without an Internet subscription,"B28003_005EA,B28003_005M,B28003_005MA"
B28003_006E,Estimate!!Total:!!No computer,"B28003_006EA,B28003_006M,B28003_006MA"


In [78]:
short_vars[short_vars['group'] == 'B08301'][['label','attributes']].sort_index() 


,label,attributes
B08301_001E,Estimate!!Total:,"B08301_001EA,B08301_001M,B08301_001MA"
B08301_002E,"Estimate!!Total:!!Car, truck, or van:","B08301_002EA,B08301_002M,B08301_002MA"
B08301_003E,"Estimate!!Total:!!Car, truck, or van:!!Drove alone","B08301_003EA,B08301_003M,B08301_003MA"
B08301_004E,"Estimate!!Total:!!Car, truck, or van:!!Carpooled:","B08301_004EA,B08301_004M,B08301_004MA"
B08301_005E,"Estimate!!Total:!!Car, truck, or van:!!Carpooled:!!In 2-person carpool","B08301_005EA,B08301_005M,B08301_005MA"
B08301_006E,"Estimate!!Total:!!Car, truck, or van:!!Carpooled:!!In 3-person carpool","B08301_006EA,B08301_006M,B08301_006MA"
B08301_007E,"Estimate!!Total:!!Car, truck, or van:!!Carpooled:!!In 4-person carpool","B08301_007EA,B08301_007M,B08301_007MA"
B08301_008E,"Estimate!!Total:!!Car, truck, or van:!!Carpooled:!!In 5- or 6-person carpool","B08301_008EA,B08301_008M,B08301_008MA"
B08301_009E,"Estimate!!Total:!!Car, truck, or van:!!Carpooled:!!In 7-or-more-person carpool","B08301_009EA,B08301_009M,B08301_009MA"
B08301_010E,Estimate!!Total:!!Public transportation (excluding taxicab):,"B08301_010EA,B08301_010M,B08301_010MA"


In [30]:
short_vars[short_vars['group'] == 'C24010'][['label','attributes']].sort_index() 

,label,attributes
C24010_001E,Estimate!!Total:,"C24010_001EA,C24010_001M,C24010_001MA"
C24010_002E,Estimate!!Total:!!Male:,"C24010_002EA,C24010_002M,C24010_002MA"
C24010_003E,"Estimate!!Total:!!Male:!!Management, business, science, and arts occupations:","C24010_003EA,C24010_003M,C24010_003MA"
C24010_004E,"Estimate!!Total:!!Male:!!Management, business, science, and arts occupations:!!Management, business, and financial occupations:","C24010_004EA,C24010_004M,C24010_004MA"
C24010_005E,"Estimate!!Total:!!Male:!!Management, business, science, and arts occupations:!!Management, business, and financial occupations:!!Management occupations","C24010_005EA,C24010_005M,C24010_005MA"
C24010_006E,"Estimate!!Total:!!Male:!!Management, business, science, and arts occupations:!!Management, business, and financial occupations:!!Business and financial operations occupations","C24010_006EA,C24010_006M,C24010_006MA"
C24010_007E,"Estimate!!Total:!!Male:!!Management, business, science, and arts occupations:!!Computer, engineering, and science occupations:","C24010_007EA,C24010_007M,C24010_007MA"
C24010_008E,"Estimate!!Total:!!Male:!!Management, business, science, and arts occupations:!!Computer, engineering, and science occupations:!!Computer and mathematical occupations","C24010_008EA,C24010_008M,C24010_008MA"
C24010_009E,"Estimate!!Total:!!Male:!!Management, business, science, and arts occupations:!!Computer, engineering, and science occupations:!!Architecture and engineering occupations","C24010_009EA,C24010_009M,C24010_009MA"
C24010_010E,"Estimate!!Total:!!Male:!!Management, business, science, and arts occupations:!!Computer, engineering, and science occupations:!!Life, physical, and social science occupations","C24010_010EA,C24010_010M,C24010_010MA"


In [31]:
short_vars[short_vars['group'] == 'C24030'][['label']].sort_index() 

,label
C24030_001E,Estimate!!Total:
C24030_002E,Estimate!!Total:!!Male:
C24030_003E,"Estimate!!Total:!!Male:!!Agriculture, forestry, fishing and hunting, and mining:"
C24030_004E,"Estimate!!Total:!!Male:!!Agriculture, forestry, fishing and hunting, and mining:!!Agriculture, forestry, fishing and hunting"
C24030_005E,"Estimate!!Total:!!Male:!!Agriculture, forestry, fishing and hunting, and mining:!!Mining, quarrying, and oil and gas extraction"
C24030_006E,Estimate!!Total:!!Male:!!Construction
C24030_007E,Estimate!!Total:!!Male:!!Manufacturing
C24030_008E,Estimate!!Total:!!Male:!!Wholesale trade
C24030_009E,Estimate!!Total:!!Male:!!Retail trade
C24030_010E,"Estimate!!Total:!!Male:!!Transportation and warehousing, and utilities:"


In [32]:
short_vars[short_vars['group'] == 'B08604'][['label']].sort_index() 


,label
B08604_001E,Estimate!!Total:


In [33]:
# CALCULATING MEDIAN HOUSEHOLD INCOME
# https://atcoordinates.info/2019/05/13/calculating-mean-income-for-groups-of-geographies-with-census-acs-data/
# https://mcdc.missouri.edu/help/measures-of-income/
# https://www.socialexplorer.com/help/faq/knowledge-base/calculating-medians
# https://s4.ad.brown.edu/Projects/Diversity/SUC/MHHINote.htm

In [34]:
# search function
# sample = censusdata.search('acs5', 2017, 'concept', 'age ')
# print(sample)

#  to see list of all tables in the ACS5:  c.acs5.tables()

In [35]:
c.acs5.tables()

[{'name': 'B17015',
  'description': 'POVERTY STATUS IN THE PAST 12 MONTHS OF FAMILIES BY FAMILY TYPE BY SOCIAL SECURITY INCOME BY SUPPLEMENTAL SECURITY INCOME (SSI) AND CASH PUBLIC ASSISTANCE INCOME',
  'variables': 'http://api.census.gov/data/2018/acs/acs5/groups/B17015.json'},
 {'name': 'B18104',
  'description': 'SEX BY AGE BY COGNITIVE DIFFICULTY',
  'variables': 'http://api.census.gov/data/2018/acs/acs5/groups/B18104.json'},
 {'name': 'B17016',
  'description': 'POVERTY STATUS IN THE PAST 12 MONTHS OF FAMILIES BY FAMILY TYPE BY WORK EXPERIENCE OF HOUSEHOLDER AND SPOUSE',
  'variables': 'http://api.census.gov/data/2018/acs/acs5/groups/B17016.json'},
 {'name': 'B18105',
  'description': 'SEX BY AGE BY AMBULATORY DIFFICULTY',
  'variables': 'http://api.census.gov/data/2018/acs/acs5/groups/B18105.json'},
 {'name': 'B17017',
  'description': 'POVERTY STATUS IN THE PAST 12 MONTHS BY HOUSEHOLD TYPE BY AGE OF HOUSEHOLDER',
  'variables': 'http://api.census.gov/data/2018/acs/acs5/groups/B

In [36]:
# censusdata.printtable(censusdata.censustable('acs5', 2020, 'B08604')) # didn't use bc got error message for this for 2020

In [37]:
# calculating median income (ACS) https://www.census.gov/quickfacts/fact/note/US/INC110221

In [38]:
# Run Census Search to retrieve data on Baltimore City, MD (all census tracts in Baltimore City)
# ***See https://api.census.gov/data/2019/acs/acs5/groups.html  for list of variables and groups for the ACS 5-year estimates***
# ex. "B23025_005E" is "unemployment count"
# The state FIPS code for MD is 24 and the FIPS code for Balt City is 510; * is to pull data for all census tracts in the 510 FIPs
census_data = c.acs5.state_county_tract(("NAME", "B11002_001E", 
                          "B19001_001E","B19013_001E", "B01003_001E", "B01002_001E",
                          "B19301_001E",
                          "B17001_002E",
                          "B23025_001E",
                          "B23025_002E",
                          "B15003_016E",               
                          "B15003_017E",
                          "B15003_021E",               
                          "B15003_022E",                
                          "B02001_002E",
                          "B02001_003E",
                          "B02001_005E",              
                          "B02001_008E",
                          "B03001_003E",
                          "B05002_013E",
                          "B08301_010E",              
                          "B25044_003E",
                          "B25044_010E",
                          "B25008_002E",
                          "B25003_001E",               
                          "B25003_002E",                                    
                          "B25003_003E",
                          "B28003_005E",
                          "B28003_006E",
                          "C17002_001E",              
                          "C24030_006E",                
                          "C24030_007E", 
                          "C24030_009E",                
                          "C24030_010E",                
                          "C24030_014E",                
                          "C24030_017E",                
                          "C24030_021E",                
                          "C24030_024E",
                          "C24030_028E",              
                          "C24030_033E",                
                          "C24030_034E",                
                          "C24030_036E",                
                          "C24030_037E",                
                          "C24030_041E",  
                          "C24030_044E",               
                          "C24030_048E",                
                          "C24030_051E",
                          "C24030_055E"),               
                          state_fips = "24",
                          county_fips = "510",
                          tract = "*")
census_pd = pd.DataFrame(census_data)
census_pd.head()      

,NAME,B11002_001E,B19001_001E,B19013_001E,B01003_001E,B01002_001E,B19301_001E,B17001_002E,B23025_001E,B23025_002E,...,C24030_036E,C24030_037E,C24030_041E,C24030_044E,C24030_048E,C24030_051E,C24030_055E,state,county,tract
0,"Census Tract 1308.05, Baltimore city, Maryland",943.0,568.0,39750.0,943.0,57.9,33640.0,171.0,874.0,461.0,...,11.0,0.0,4.0,52.0,94.0,4.0,13.0,24,510,130805
1,"Census Tract 2101, Baltimore city, Maryland",2532.0,967.0,63558.0,2532.0,31.1,30254.0,433.0,2102.0,1653.0,...,90.0,33.0,39.0,57.0,245.0,144.0,52.0,24,510,210100
2,"Census Tract 2707.01, Baltimore city, Maryland",2468.0,1139.0,36994.0,2468.0,31.9,28810.0,461.0,2045.0,1708.0,...,29.0,25.0,70.0,142.0,481.0,32.0,88.0,24,510,270701
3,"Census Tract 1901, Baltimore city, Maryland",2108.0,808.0,21630.0,2131.0,34.9,17518.0,1067.0,1529.0,732.0,...,43.0,33.0,5.0,59.0,146.0,6.0,11.0,24,510,190100
4,"Census Tract 1902, Baltimore city, Maryland",1886.0,698.0,32500.0,1886.0,34.4,22552.0,591.0,1481.0,1035.0,...,130.0,20.0,14.0,30.0,127.0,113.0,35.0,24,510,190200


In [39]:
# check variables that are similar
check = c.acs5.state_county(("NAME", 
                          "B23025_001E",
                          "B23025_002E",
                          "B23025_004E",
                          "B23025_005E"),               
                          state_fips = "24",
                          county_fips = "510",
                          )
check_df = pd.DataFrame(check)
check_df      

,NAME,B23025_001E,B23025_002E,B23025_004E,B23025_005E,state,county
0,"Baltimore city, Maryland",498965.0,307536.0,279034.0,27963.0,24,510


In [40]:
short_vars[short_vars['group'] == 'B23025'][['label','attributes']].sort_index() 

,label,attributes
B23025_001E,Estimate!!Total:,"B23025_001EA,B23025_001M,B23025_001MA"
B23025_002E,Estimate!!Total:!!In labor force:,"B23025_002EA,B23025_002M,B23025_002MA"
B23025_003E,Estimate!!Total:!!In labor force:!!Civilian labor force:,"B23025_003EA,B23025_003M,B23025_003MA"
B23025_004E,Estimate!!Total:!!In labor force:!!Civilian labor force:!!Employed,"B23025_004EA,B23025_004M,B23025_004MA"
B23025_005E,Estimate!!Total:!!In labor force:!!Civilian labor force:!!Unemployed,"B23025_005EA,B23025_005M,B23025_005MA"
B23025_006E,Estimate!!Total:!!In labor force:!!Armed Forces,"B23025_006EA,B23025_006M,B23025_006MA"
B23025_007E,Estimate!!Total:!!Not in labor force,"B23025_007EA,B23025_007M,B23025_007MA"


In [41]:
# check variables 
check2 = c.acs5.state_county(("NAME", 
                          "B08301_001E",
                          "B08301_010E"),               
                          state_fips = "24",
                          county_fips = "510",
                          )
check_df2 = pd.DataFrame(check2)
check_df2     

,NAME,B08301_001E,B08301_010E,state,county
0,"Baltimore city, Maryland",274479.0,49309.0,24,510


In [42]:
short_vars[short_vars['group'] == 'B08301'][['label','attributes']].sort_index() 

,label,attributes
B08301_001E,Estimate!!Total:,"B08301_001EA,B08301_001M,B08301_001MA"
B08301_002E,"Estimate!!Total:!!Car, truck, or van:","B08301_002EA,B08301_002M,B08301_002MA"
B08301_003E,"Estimate!!Total:!!Car, truck, or van:!!Drove alone","B08301_003EA,B08301_003M,B08301_003MA"
B08301_004E,"Estimate!!Total:!!Car, truck, or van:!!Carpooled:","B08301_004EA,B08301_004M,B08301_004MA"
B08301_005E,"Estimate!!Total:!!Car, truck, or van:!!Carpooled:!!In 2-person carpool","B08301_005EA,B08301_005M,B08301_005MA"
B08301_006E,"Estimate!!Total:!!Car, truck, or van:!!Carpooled:!!In 3-person carpool","B08301_006EA,B08301_006M,B08301_006MA"
B08301_007E,"Estimate!!Total:!!Car, truck, or van:!!Carpooled:!!In 4-person carpool","B08301_007EA,B08301_007M,B08301_007MA"
B08301_008E,"Estimate!!Total:!!Car, truck, or van:!!Carpooled:!!In 5- or 6-person carpool","B08301_008EA,B08301_008M,B08301_008MA"
B08301_009E,"Estimate!!Total:!!Car, truck, or van:!!Carpooled:!!In 7-or-more-person carpool","B08301_009EA,B08301_009M,B08301_009MA"
B08301_010E,Estimate!!Total:!!Public transportation (excluding taxicab):,"B08301_010EA,B08301_010M,B08301_010MA"


In [43]:
census_pd.columns

Index(['NAME', 'B11002_001E', 'B19001_001E', 'B19013_001E', 'B01003_001E',
       'B01002_001E', 'B19301_001E', 'B17001_002E', 'B23025_001E',
       'B23025_002E', 'B15003_016E', 'B15003_017E', 'B15003_021E',
       'B15003_022E', 'B02001_002E', 'B02001_003E', 'B02001_005E',
       'B02001_008E', 'B03001_003E', 'B05002_013E', 'B08301_010E',
       'B25044_003E', 'B25044_010E', 'B25008_002E', 'B25003_001E',
       'B25003_002E', 'B25003_003E', 'B28003_005E', 'B28003_006E',
       'C17002_001E', 'C24030_006E', 'C24030_007E', 'C24030_009E',
       'C24030_010E', 'C24030_014E', 'C24030_017E', 'C24030_021E',
       'C24030_024E', 'C24030_028E', 'C24030_033E', 'C24030_034E',
       'C24030_036E', 'C24030_037E', 'C24030_041E', 'C24030_044E',
       'C24030_048E', 'C24030_051E', 'C24030_055E', 'state', 'county',
       'tract'],
      dtype='object')

In [44]:
# Pull values from the ACS 5 yr. census codes/ variables listed and save them in the variable "age_data"

age_data = c.acs5.state_county_tract(("NAME", "B01001_003E",                                    
                          "B01001_004E",
                          "B01001_005E",
                          "B01001_006E",
                          "B01001_007E",
                          "B01001_008E",
                          "B01001_009E",
                          "B01001_010E",
                          "B01001_011E",
                          "B01001_012E",
                          "B01001_013E",
                          "B01001_014E",
                          "B01001_015E",
                          "B01001_016E",            
                          "B01001_017E",
                          "B01001_018E",
                          "B01001_019E",
                          "B01001_020E",
                          "B01001_021E",
                          "B01001_022E",
                          "B01001_023E",
                          "B01001_024E",
                          "B01001_025E",
                          "B01001_027E",
                          "B01001_028E",
                          "B01001_029E",
                          "B01001_030E",
                          "B01001_031E",
                          "B01001_032E",
                          "B01001_033E",
                          "B01001_034E",
                          "B01001_035E",
                          "B01001_036E",
                          "B01001_037E",
                          "B01001_038E",
                          "B01001_039E",            
                          "B01001_040E",
                          "B01001_041E",
                          "B01001_042E",
                          "B01001_043E",
                          "B01001_044E",
                          "B01001_045E",
                          "B01001_046E",
                          "B01001_047E",
                          "B01001_048E",
                          "B01001_049E"),          
                          state_fips = "24",
                          county_fips = "510",
                          tract = "*")
age_pd = pd.DataFrame(age_data)
age_pd.head()      

,NAME,B01001_003E,B01001_004E,B01001_005E,B01001_006E,B01001_007E,B01001_008E,B01001_009E,B01001_010E,B01001_011E,...,B01001_043E,B01001_044E,B01001_045E,B01001_046E,B01001_047E,B01001_048E,B01001_049E,state,county,tract
0,"Census Tract 1308.05, Baltimore city, Maryland",18.0,0.0,9.0,0.0,4.0,17.0,0.0,0.0,10.0,...,17.0,71.0,40.0,34.0,18.0,30.0,32.0,24,510,130805
1,"Census Tract 2101, Baltimore city, Maryland",48.0,52.0,32.0,31.0,64.0,8.0,9.0,39.0,129.0,...,26.0,10.0,14.0,54.0,5.0,22.0,0.0,24,510,210100
2,"Census Tract 2707.01, Baltimore city, Maryland",145.0,88.0,24.0,33.0,0.0,0.0,19.0,41.0,187.0,...,47.0,10.0,13.0,7.0,35.0,16.0,5.0,24,510,270701
3,"Census Tract 1901, Baltimore city, Maryland",55.0,66.0,43.0,32.0,23.0,0.0,0.0,0.0,75.0,...,8.0,24.0,7.0,25.0,53.0,38.0,27.0,24,510,190100
4,"Census Tract 1902, Baltimore city, Maryland",64.0,155.0,22.0,54.0,17.0,0.0,0.0,14.0,74.0,...,6.0,9.0,32.0,12.0,26.0,0.0,0.0,24,510,190200


In [45]:
# Sum columns of age groups that are under 18 years old for male and female and add new column "Pop. <18 years"
columns_under18 = ["B01001_003E",                                    
                   "B01001_004E",
                   "B01001_005E",
                   "B01001_006E",
                   "B01001_027E",
                   "B01001_028E",
                   "B01001_029E",
                   "B01001_030E"]
age_pd['Pop. <18 years']= age_pd[columns_under18].sum(axis=1)
age_pd.head()

,NAME,B01001_003E,B01001_004E,B01001_005E,B01001_006E,B01001_007E,B01001_008E,B01001_009E,B01001_010E,B01001_011E,...,B01001_044E,B01001_045E,B01001_046E,B01001_047E,B01001_048E,B01001_049E,state,county,tract,Pop. <18 years
0,"Census Tract 1308.05, Baltimore city, Maryland",18.0,0.0,9.0,0.0,4.0,17.0,0.0,0.0,10.0,...,71.0,40.0,34.0,18.0,30.0,32.0,24,510,130805,83.0
1,"Census Tract 2101, Baltimore city, Maryland",48.0,52.0,32.0,31.0,64.0,8.0,9.0,39.0,129.0,...,10.0,14.0,54.0,5.0,22.0,0.0,24,510,210100,511.0
2,"Census Tract 2707.01, Baltimore city, Maryland",145.0,88.0,24.0,33.0,0.0,0.0,19.0,41.0,187.0,...,10.0,13.0,7.0,35.0,16.0,5.0,24,510,270701,460.0
3,"Census Tract 1901, Baltimore city, Maryland",55.0,66.0,43.0,32.0,23.0,0.0,0.0,0.0,75.0,...,24.0,7.0,25.0,53.0,38.0,27.0,24,510,190100,650.0
4,"Census Tract 1902, Baltimore city, Maryland",64.0,155.0,22.0,54.0,17.0,0.0,0.0,14.0,74.0,...,9.0,32.0,12.0,26.0,0.0,0.0,24,510,190200,446.0


In [46]:
# Sum columns of age groups that are 18-64 years old (working age) for male and female and add new column "Pop. working age"
columns_working_age = [                                    
                   "B01001_007E",
                   "B01001_008E",
                   "B01001_009E",
                   "B01001_010E",
                   "B01001_011E",
                   "B01001_012E",   
                   "B01001_013E",
                   "B01001_014E",    
                   "B01001_015E",    
                   "B01001_016E",   
                   "B01001_017E", 
                   "B01001_018E",    
                   "B01001_019E",    
                   "B01001_031E",
                   "B01001_032E",
                   "B01001_033E",
                   "B01001_034E",
                   "B01001_035E",   
                   "B01001_036E",
                   "B01001_037E",    
                   "B01001_038E",    
                   "B01001_039E",   
                   "B01001_040E", 
                   "B01001_041E",    
                   "B01001_042E",    
                   "B01001_043E"]    
                           
age_pd['Pop. working age']= age_pd[columns_working_age].sum(axis=1)
age_pd.head()    
            
               


,NAME,B01001_003E,B01001_004E,B01001_005E,B01001_006E,B01001_007E,B01001_008E,B01001_009E,B01001_010E,B01001_011E,...,B01001_045E,B01001_046E,B01001_047E,B01001_048E,B01001_049E,state,county,tract,Pop. <18 years,Pop. working age
0,"Census Tract 1308.05, Baltimore city, Maryland",18.0,0.0,9.0,0.0,4.0,17.0,0.0,0.0,10.0,...,40.0,34.0,18.0,30.0,32.0,24,510,130805,83.0,486.0
1,"Census Tract 2101, Baltimore city, Maryland",48.0,52.0,32.0,31.0,64.0,8.0,9.0,39.0,129.0,...,14.0,54.0,5.0,22.0,0.0,24,510,210100,511.0,1862.0
2,"Census Tract 2707.01, Baltimore city, Maryland",145.0,88.0,24.0,33.0,0.0,0.0,19.0,41.0,187.0,...,13.0,7.0,35.0,16.0,5.0,24,510,270701,460.0,1882.0
3,"Census Tract 1901, Baltimore city, Maryland",55.0,66.0,43.0,32.0,23.0,0.0,0.0,0.0,75.0,...,7.0,25.0,53.0,38.0,27.0,24,510,190100,650.0,1201.0
4,"Census Tract 1902, Baltimore city, Maryland",64.0,155.0,22.0,54.0,17.0,0.0,0.0,14.0,74.0,...,32.0,12.0,26.0,0.0,0.0,24,510,190200,446.0,1293.0


In [47]:
# Sum columns of age groups that are 65+ years old for male and female and add new column "Pop. 65+ years"
columns_senior = ["B01001_020E",
                  "B01001_021E",
                  "B01001_022E",
                  "B01001_023E",                                    
                  "B01001_024E",
                  "B01001_025E",
                  "B01001_044E",
                  "B01001_045E",
                  "B01001_046E",
                  "B01001_047E",                                    
                  "B01001_048E",
                  "B01001_049E"]               
                          
age_pd['Pop. 65+ years']= age_pd[columns_senior].sum(axis=1)
age_pd.head()                           

,NAME,B01001_003E,B01001_004E,B01001_005E,B01001_006E,B01001_007E,B01001_008E,B01001_009E,B01001_010E,B01001_011E,...,B01001_046E,B01001_047E,B01001_048E,B01001_049E,state,county,tract,Pop. <18 years,Pop. working age,Pop. 65+ years
0,"Census Tract 1308.05, Baltimore city, Maryland",18.0,0.0,9.0,0.0,4.0,17.0,0.0,0.0,10.0,...,34.0,18.0,30.0,32.0,24,510,130805,83.0,486.0,374.0
1,"Census Tract 2101, Baltimore city, Maryland",48.0,52.0,32.0,31.0,64.0,8.0,9.0,39.0,129.0,...,54.0,5.0,22.0,0.0,24,510,210100,511.0,1862.0,159.0
2,"Census Tract 2707.01, Baltimore city, Maryland",145.0,88.0,24.0,33.0,0.0,0.0,19.0,41.0,187.0,...,7.0,35.0,16.0,5.0,24,510,270701,460.0,1882.0,126.0
3,"Census Tract 1901, Baltimore city, Maryland",55.0,66.0,43.0,32.0,23.0,0.0,0.0,0.0,75.0,...,25.0,53.0,38.0,27.0,24,510,190100,650.0,1201.0,280.0
4,"Census Tract 1902, Baltimore city, Maryland",64.0,155.0,22.0,54.0,17.0,0.0,0.0,14.0,74.0,...,12.0,26.0,0.0,0.0,24,510,190200,446.0,1293.0,147.0


In [48]:
age_temp = age_pd[[ "state", "county", "tract", "Pop. <18 years", "Pop. working age", "Pop. 65+ years"]]
age_temp.head()

,state,county,tract,Pop. <18 years,Pop. working age,Pop. 65+ years
0,24,510,130805,83.0,486.0,374.0
1,24,510,210100,511.0,1862.0,159.0
2,24,510,270701,460.0,1882.0,126.0
3,24,510,190100,650.0,1201.0,280.0
4,24,510,190200,446.0,1293.0,147.0


In [49]:
age_temp.dtypes

state                object
county               object
tract                object
Pop. <18 years      float64
Pop. working age    float64
Pop. 65+ years      float64
dtype: object

In [50]:
age_final = age_temp.copy()

In [51]:
# Create "GEOID" column, that we will later merge tables on
age_final["GEOID1"] = age_final[['state','county', 'tract']].agg(''.join,axis=1) 
age_final

,state,county,tract,Pop. <18 years,Pop. working age,Pop. 65+ years,GEOID1
0,24,510,130805,83.0,486.0,374.0,24510130805
1,24,510,210100,511.0,1862.0,159.0,24510210100
2,24,510,270701,460.0,1882.0,126.0,24510270701
3,24,510,190100,650.0,1201.0,280.0,24510190100
4,24,510,190200,446.0,1293.0,147.0,24510190200
5,24,510,220100,380.0,3164.0,791.0,24510220100
6,24,510,230300,16.0,1316.0,99.0,24510230300
7,24,510,250207,676.0,1375.0,257.0,24510250207
8,24,510,250303,543.0,1533.0,294.0,24510250303
9,24,510,260202,1205.0,3857.0,467.0,24510260202


In [52]:
# remove extraneous columns
age_final = age_final.drop(["state", "county", "tract"], axis=1)
age_final.head()

,Pop. <18 years,Pop. working age,Pop. 65+ years,GEOID1
0,83.0,486.0,374.0,24510130805
1,511.0,1862.0,159.0,24510210100
2,460.0,1882.0,126.0,24510270701
3,650.0,1201.0,280.0,24510190100
4,446.0,1293.0,147.0,24510190200


In [53]:
# Did not add in daytime population - will use ESRI business analyst for this 
# See https://www.census.gov/topics/employment/commuting/guidance/calculations.html
# "commuter-adjusted daytime population estimates" =    
#         total resident population + total workers working in area - total workers living in area

# For "Workers in Workplace Geography," see https://www.census.gov/topics/employment/commuting/guidance/calculations.html
# "Total workers working in area:
# B08604 Total Workers for Workplace Geography
# B08604 is only available for data years 2011 and after. 
# The tables for workplace geography are only available for the following geographic summary levels: States; 
# Counties; Places; County Subdivisions in selected states (not MD); Combined Statistical Areas; Metropolitan 
# and Micropolitan Statistical Areas, and their associated Metropolitan Divisions and Principal Cities; 

census_data_workers = c.acs5.state_county(("NAME", 
                          "B08604_001E"),               
                          state_fips = "24",
                          county_fips = "510") 

# convert to dataframe
workers_df = pd.DataFrame(census_data_workers)
workers_df
                         

,NAME,B08604_001E,state,county
0,"Baltimore city, Maryland",382638.0,24,510


In [54]:
# Create Geographic Identifier ("GEOID") for each census tract by adding state fips code + county fips code + census tract code
# see https://www.census.gov/programs-surveys/geography/guidance/geo-identifiers.html#:~:text=The%20full%20GEOID%20for%20many,codes%2C%20in%20which%20they%20nest.
census_pd["GEOID"] = census_pd['state'] + census_pd['county'] + census_pd['tract']
census_pd

,NAME,B11002_001E,B19001_001E,B19013_001E,B01003_001E,B01002_001E,B19301_001E,B17001_002E,B23025_001E,B23025_002E,...,C24030_037E,C24030_041E,C24030_044E,C24030_048E,C24030_051E,C24030_055E,state,county,tract,GEOID
0,"Census Tract 1308.05, Baltimore city, Maryland",943.0,568.0,39750.0,943.0,57.9,33640.0,171.0,874.0,461.0,...,0.0,4.0,52.0,94.0,4.0,13.0,24,510,130805,24510130805
1,"Census Tract 2101, Baltimore city, Maryland",2532.0,967.0,63558.0,2532.0,31.1,30254.0,433.0,2102.0,1653.0,...,33.0,39.0,57.0,245.0,144.0,52.0,24,510,210100,24510210100
2,"Census Tract 2707.01, Baltimore city, Maryland",2468.0,1139.0,36994.0,2468.0,31.9,28810.0,461.0,2045.0,1708.0,...,25.0,70.0,142.0,481.0,32.0,88.0,24,510,270701,24510270701
3,"Census Tract 1901, Baltimore city, Maryland",2108.0,808.0,21630.0,2131.0,34.9,17518.0,1067.0,1529.0,732.0,...,33.0,5.0,59.0,146.0,6.0,11.0,24,510,190100,24510190100
4,"Census Tract 1902, Baltimore city, Maryland",1886.0,698.0,32500.0,1886.0,34.4,22552.0,591.0,1481.0,1035.0,...,20.0,14.0,30.0,127.0,113.0,35.0,24,510,190200,24510190200
5,"Census Tract 2201, Baltimore city, Maryland",4292.0,2264.0,75652.0,4335.0,33.9,63475.0,449.0,4022.0,2843.0,...,12.0,181.0,176.0,630.0,121.0,97.0,24,510,220100,24510220100
6,"Census Tract 2303, Baltimore city, Maryland",1398.0,730.0,106806.0,1431.0,30.5,61426.0,120.0,1418.0,1190.0,...,0.0,52.0,116.0,219.0,14.0,76.0,24,510,230300,24510230300
7,"Census Tract 2502.07, Baltimore city, Maryland",2308.0,817.0,34081.0,2308.0,31.1,19115.0,700.0,1708.0,963.0,...,0.0,36.0,78.0,237.0,51.0,51.0,24,510,250207,24510250207
8,"Census Tract 2503.03, Baltimore city, Maryland",2370.0,872.0,38750.0,2370.0,36.4,21467.0,648.0,1859.0,1144.0,...,44.0,17.0,26.0,108.0,98.0,67.0,24,510,250303,24510250303
9,"Census Tract 2602.02, Baltimore city, Maryland",5524.0,2336.0,39283.0,5529.0,33.4,21522.0,920.0,4449.0,2996.0,...,0.0,74.0,189.0,707.0,91.0,124.0,24,510,260202,24510260202


In [55]:
# "Number of rows" = # of census tracts in the dataframe
print("Number of rows, columns: ", census_pd.shape)

Number of rows, columns:  (200, 52)


In [56]:
# Replace the census variable codes (such as "B19013_001E") in the dataframe with text so it's understandable
census_pd = census_pd.rename(columns={"B19001_001E": "Total households",
                                      "B01003_001E": "Population",
                                      "tract": "Census Tract", "C17002_001E": "poverty", 
                                      "B08301_010E": "Commute_to_work_public_transportation",       
                                      "B01002_001E": "Median age",
                                      "B11002_001E": "Household population",
                                      "B19013_001E": "Median household income",
                                      "B19301_001E": "Per capita income", 
                                      "B17001_002E": "Poverty count",
                                      "B23025_004E": "# employed, age 16+",
                                      "B23025_005E": "Unemployment count",
                                      "B15003_016E": "# persons 12th grade, no diploma",
                                      "B15003_017E": "# persons age 25+ graduated high school",
                                      "B15003_021E": "# persons Associate's degree",
                                      "B15003_022E": "# persons age 25+ with Bachelor's degree",
                                      "B02001_002E": "Pop. white",
                                      "B02001_003E": "Pop. Black",
                                      "B02001_005E": "Pop. Asian",        
                                      "B02001_008E": "Pop. two or more races",
                                      "B03001_003E": "Pop. Hispanic origin",
                                      "B05002_013E": "# Foreign-born",
                                      "B23025_001E": "Pop_16_yrs_and_over",
                                      "B23025_002E": "In_labor_force",
                                      "B25008_002E": "Total pop. in occupied housing units by tenure",
                                      "B25003_001E": "Total occupied units",
                                      "B25003_002E": "Total owner-occupied units",
                                      "B25003_003E": "Total renter-occupied units",
                                      "B25044_003E": "No_vehicle_available_owneroccupied_unit",
                                      "B25044_010E": "No_vehicle_available_renteroccupied_unit",
                                      "B28003_005E": "# households with computer, no internet subscription",
                                      "B28003_006E": "# households no computer",
                                      
                                      "C24030_006E": "ConstructionM", 
                                      "C24030_007E": "ManufacturingM", 
                                      "C24030_009E": "Retail_TradeM",
                                      "C24030_010E": "Transportation_warehousing_utilitiesM",
                                      "C24030_014E": "Finance_insurance_realestateM",               
                                      "C24030_017E": "Professional_scientific_mgmt_administrativeM",              
                                      "C24030_021E": "Educational_healthcare_socialM",                
                                      "C24030_024E": "Accommodations_foodservices_arts_entertainmentM",
                                      "C24030_028E": "Public_administrationM",
                                      "C24030_033E": "ConstructionF",                
                                      "C24030_034E": "ManufacturingF",
                                      "C24030_036E": "Retail_TradeF",                
                                      "C24030_037E": "Transportation_warehousing_utilitiesF",                
                                      "C24030_041E": "Finance_insurance_realestateF",  
                                      "C24030_044E": "Professional_scientific_mgmt_administrativeF",               
                                      "C24030_048E": "Educational_healthcare_socialF",
                                      "C24030_051E": "Accommodations_foodservices_arts_entertainmentF",
                                      "C24030_055E": "Public_administrationF",
                                      "NAME": "Name", "state": "State", "GEOID": "GEOID"
                                     })       
census_pd.head()                              



,Name,Household population,Total households,Median household income,Population,Median age,Per capita income,Poverty count,Pop_16_yrs_and_over,In_labor_force,...,Transportation_warehousing_utilitiesF,Finance_insurance_realestateF,Professional_scientific_mgmt_administrativeF,Educational_healthcare_socialF,Accommodations_foodservices_arts_entertainmentF,Public_administrationF,State,county,Census Tract,GEOID
0,"Census Tract 1308.05, Baltimore city, Maryland",943.0,568.0,39750.0,943.0,57.9,33640.0,171.0,874.0,461.0,...,0.0,4.0,52.0,94.0,4.0,13.0,24,510,130805,24510130805
1,"Census Tract 2101, Baltimore city, Maryland",2532.0,967.0,63558.0,2532.0,31.1,30254.0,433.0,2102.0,1653.0,...,33.0,39.0,57.0,245.0,144.0,52.0,24,510,210100,24510210100
2,"Census Tract 2707.01, Baltimore city, Maryland",2468.0,1139.0,36994.0,2468.0,31.9,28810.0,461.0,2045.0,1708.0,...,25.0,70.0,142.0,481.0,32.0,88.0,24,510,270701,24510270701
3,"Census Tract 1901, Baltimore city, Maryland",2108.0,808.0,21630.0,2131.0,34.9,17518.0,1067.0,1529.0,732.0,...,33.0,5.0,59.0,146.0,6.0,11.0,24,510,190100,24510190100
4,"Census Tract 1902, Baltimore city, Maryland",1886.0,698.0,32500.0,1886.0,34.4,22552.0,591.0,1481.0,1035.0,...,20.0,14.0,30.0,127.0,113.0,35.0,24,510,190200,24510190200


In [ ]:
B08301

In [ ]:
# check variables that are similar
check = c.acs5.state_county(("NAME", 
                          "B08301_001E",
                          "B23025_002E",
                          "B23025_004E",
                          "B23025_005E"),               
                          state_fips = "24",
                          county_fips = "510",
                          )
check_df = pd.DataFrame(check)
check_df      

In [57]:
# Get the sum of the column "poverty" (table  "C17002_001E")
pov_total = census_pd['poverty'].sum()
print(pov_total)

591942.0


In [58]:
# Get the sum of the column "Poverty count" (table "B17001_002E" )
pov_count_total = census_pd['Poverty count'].sum()
print(pov_count_total)

128829.0


In [59]:
census_pd.columns

Index(['Name', 'Household population', 'Total households',
       'Median household income', 'Population', 'Median age',
       'Per capita income', 'Poverty count', 'Pop_16_yrs_and_over',
       'In_labor_force', '# persons 12th grade, no diploma',
       '# persons age 25+ graduated high school',
       '# persons Associate's degree',
       '# persons age 25+ with Bachelor's degree', 'Pop. white', 'Pop. Black',
       'Pop. Asian', 'Pop. two or more races', 'Pop. Hispanic origin',
       '# Foreign-born', 'Commute_to_work_public_transportation',
       'No_vehicle_available_owneroccupied_unit',
       'No_vehicle_available_renteroccupied_unit',
       'Total pop. in occupied housing units by tenure',
       'Total occupied units', 'Total owner-occupied units',
       'Total renter-occupied units',
       '# households with computer, no internet subscription',
       '# households no computer', 'poverty', 'ConstructionM',
       'ManufacturingM', 'Retail_TradeM',
       'Transportati

In [60]:
# Create a new column for each job industry by adding male and female values for that industry together 
census_pd["Construction"] = census_pd.apply(lambda row: row.ConstructionM + row.ConstructionF, axis=1)
census_pd.columns

Index(['Name', 'Household population', 'Total households',
       'Median household income', 'Population', 'Median age',
       'Per capita income', 'Poverty count', 'Pop_16_yrs_and_over',
       'In_labor_force', '# persons 12th grade, no diploma',
       '# persons age 25+ graduated high school',
       '# persons Associate's degree',
       '# persons age 25+ with Bachelor's degree', 'Pop. white', 'Pop. Black',
       'Pop. Asian', 'Pop. two or more races', 'Pop. Hispanic origin',
       '# Foreign-born', 'Commute_to_work_public_transportation',
       'No_vehicle_available_owneroccupied_unit',
       'No_vehicle_available_renteroccupied_unit',
       'Total pop. in occupied housing units by tenure',
       'Total occupied units', 'Total owner-occupied units',
       'Total renter-occupied units',
       '# households with computer, no internet subscription',
       '# households no computer', 'poverty', 'ConstructionM',
       'ManufacturingM', 'Retail_TradeM',
       'Transportati

In [61]:
#census_pd.dtypes

In [62]:
census_pd["Manufacturing"] = census_pd.apply(lambda row: row.ManufacturingM + row.ManufacturingF, axis=1)

In [63]:
census_pd["Retail Trade"] = census_pd.apply(lambda row: row.Retail_TradeM + row.Retail_TradeF, axis=1)

In [64]:
census_pd["Transportation, warehousing, utilities"] = census_pd.apply(lambda row: row.Transportation_warehousing_utilitiesM + row.Transportation_warehousing_utilitiesF, axis=1)

In [65]:
census_pd["Finance, insurance, real estate"] = census_pd.apply(lambda row: row.Finance_insurance_realestateM + row.Finance_insurance_realestateF, axis=1)


In [66]:
census_pd["Professional, scientific, mgmt, administrative"] = census_pd.apply(lambda row: row.Professional_scientific_mgmt_administrativeM + row.Professional_scientific_mgmt_administrativeF, axis=1)


In [67]:
census_pd["Educational services, health care, social assistance"] = census_pd.apply(lambda row: row.Educational_healthcare_socialM + row.Educational_healthcare_socialF, axis=1)


In [68]:
census_pd["Accommodations, food services, arts, entertainment"] = census_pd.apply(lambda row: row.Accommodations_foodservices_arts_entertainmentM + row.Accommodations_foodservices_arts_entertainmentF, axis=1)

In [69]:
census_pd["Public administration"] = census_pd.apply(lambda row: row.Public_administrationM + row.Public_administrationF, axis=1)

In [70]:
census_pd.columns

Index(['Name', 'Household population', 'Total households',
       'Median household income', 'Population', 'Median age',
       'Per capita income', 'Poverty count', 'Pop_16_yrs_and_over',
       'In_labor_force', '# persons 12th grade, no diploma',
       '# persons age 25+ graduated high school',
       '# persons Associate's degree',
       '# persons age 25+ with Bachelor's degree', 'Pop. white', 'Pop. Black',
       'Pop. Asian', 'Pop. two or more races', 'Pop. Hispanic origin',
       '# Foreign-born', 'Commute_to_work_public_transportation',
       'No_vehicle_available_owneroccupied_unit',
       'No_vehicle_available_renteroccupied_unit',
       'Total pop. in occupied housing units by tenure',
       'Total occupied units', 'Total owner-occupied units',
       'Total renter-occupied units',
       '# households with computer, no internet subscription',
       '# households no computer', 'poverty', 'ConstructionM',
       'ManufacturingM', 'Retail_TradeM',
       'Transportati

In [71]:
# remove extraneous columns
census_pd = census_pd.drop(['ConstructionM', 'ManufacturingM',
       'Retail_TradeM', 'Transportation_warehousing_utilitiesM',
       'Finance_insurance_realestateM',
       'Professional_scientific_mgmt_administrativeM',
       'Educational_healthcare_socialM',
       'Accommodations_foodservices_arts_entertainmentM',
       'Public_administrationM', 'ConstructionF', 'ManufacturingF',
       'Retail_TradeF', 'Transportation_warehousing_utilitiesF',
       'Finance_insurance_realestateF',
       'Professional_scientific_mgmt_administrativeF',
       'Educational_healthcare_socialF',
       'Accommodations_foodservices_arts_entertainmentF',
       'Public_administrationF', 'State', 'county'], axis=1)
census_pd.columns

Index(['Name', 'Household population', 'Total households',
       'Median household income', 'Population', 'Median age',
       'Per capita income', 'Poverty count', 'Pop_16_yrs_and_over',
       'In_labor_force', '# persons 12th grade, no diploma',
       '# persons age 25+ graduated high school',
       '# persons Associate's degree',
       '# persons age 25+ with Bachelor's degree', 'Pop. white', 'Pop. Black',
       'Pop. Asian', 'Pop. two or more races', 'Pop. Hispanic origin',
       '# Foreign-born', 'Commute_to_work_public_transportation',
       'No_vehicle_available_owneroccupied_unit',
       'No_vehicle_available_renteroccupied_unit',
       'Total pop. in occupied housing units by tenure',
       'Total occupied units', 'Total owner-occupied units',
       'Total renter-occupied units',
       '# households with computer, no internet subscription',
       '# households no computer', 'poverty', 'Census Tract', 'GEOID',
       'Construction', 'Manufacturing', 'Retail Trade'

In [72]:
# Calculate the number of unique census tracts in the DataFrame
tract_count = len(census_pd["Census Tract"].unique())
tract_count

200

In [73]:
# merge the census_pd dataframe with the age_final dataframe on the common column "GEOID"
census_joined2 = pd.concat([census_pd, age_final], axis=1)
census_joined2

,Name,Household population,Total households,Median household income,Population,Median age,Per capita income,Poverty count,Pop_16_yrs_and_over,In_labor_force,...,"Transportation, warehousing, utilities","Finance, insurance, real estate","Professional, scientific, mgmt, administrative","Educational services, health care, social assistance","Accommodations, food services, arts, entertainment",Public administration,Pop. <18 years,Pop. working age,Pop. 65+ years,GEOID1
0,"Census Tract 1308.05, Baltimore city, Maryland",943.0,568.0,39750.0,943.0,57.9,33640.0,171.0,874.0,461.0,...,20.0,16.0,74.0,117.0,25.0,49.0,83.0,486.0,374.0,24510130805
1,"Census Tract 2101, Baltimore city, Maryland",2532.0,967.0,63558.0,2532.0,31.1,30254.0,433.0,2102.0,1653.0,...,95.0,79.0,161.0,355.0,230.0,124.0,511.0,1862.0,159.0,24510210100
2,"Census Tract 2707.01, Baltimore city, Maryland",2468.0,1139.0,36994.0,2468.0,31.9,28810.0,461.0,2045.0,1708.0,...,130.0,70.0,276.0,578.0,56.0,112.0,460.0,1882.0,126.0,24510270701
3,"Census Tract 1901, Baltimore city, Maryland",2108.0,808.0,21630.0,2131.0,34.9,17518.0,1067.0,1529.0,732.0,...,48.0,15.0,113.0,161.0,68.0,39.0,650.0,1201.0,280.0,24510190100
4,"Census Tract 1902, Baltimore city, Maryland",1886.0,698.0,32500.0,1886.0,34.4,22552.0,591.0,1481.0,1035.0,...,109.0,53.0,103.0,182.0,198.0,49.0,446.0,1293.0,147.0,24510190200
5,"Census Tract 2201, Baltimore city, Maryland",4292.0,2264.0,75652.0,4335.0,33.9,63475.0,449.0,4022.0,2843.0,...,63.0,319.0,535.0,819.0,301.0,166.0,380.0,3164.0,791.0,24510220100
6,"Census Tract 2303, Baltimore city, Maryland",1398.0,730.0,106806.0,1431.0,30.5,61426.0,120.0,1418.0,1190.0,...,8.0,112.0,256.0,310.0,77.0,130.0,16.0,1316.0,99.0,24510230300
7,"Census Tract 2502.07, Baltimore city, Maryland",2308.0,817.0,34081.0,2308.0,31.1,19115.0,700.0,1708.0,963.0,...,101.0,36.0,113.0,281.0,95.0,65.0,676.0,1375.0,257.0,24510250207
8,"Census Tract 2503.03, Baltimore city, Maryland",2370.0,872.0,38750.0,2370.0,36.4,21467.0,648.0,1859.0,1144.0,...,86.0,17.0,139.0,115.0,140.0,67.0,543.0,1533.0,294.0,24510250303
9,"Census Tract 2602.02, Baltimore city, Maryland",5524.0,2336.0,39283.0,5529.0,33.4,21522.0,920.0,4449.0,2996.0,...,218.0,90.0,220.0,912.0,180.0,199.0,1205.0,3857.0,467.0,24510260202


In [74]:
census_joined2.columns

Index(['Name', 'Household population', 'Total households',
       'Median household income', 'Population', 'Median age',
       'Per capita income', 'Poverty count', 'Pop_16_yrs_and_over',
       'In_labor_force', '# persons 12th grade, no diploma',
       '# persons age 25+ graduated high school',
       '# persons Associate's degree',
       '# persons age 25+ with Bachelor's degree', 'Pop. white', 'Pop. Black',
       'Pop. Asian', 'Pop. two or more races', 'Pop. Hispanic origin',
       '# Foreign-born', 'Commute_to_work_public_transportation',
       'No_vehicle_available_owneroccupied_unit',
       'No_vehicle_available_renteroccupied_unit',
       'Total pop. in occupied housing units by tenure',
       'Total occupied units', 'Total owner-occupied units',
       'Total renter-occupied units',
       '# households with computer, no internet subscription',
       '# households no computer', 'poverty', 'Census Tract', 'GEOID',
       'Construction', 'Manufacturing', 'Retail Trade'

In [75]:
# see the values in the "GEOID" column to make sure it matches with tracts in "NAME" column (before we delete the "GEOID1" column)
#list = census_joined['GEOID'].tolist()
#print(list)


In [76]:
# Change order of columns in DataFrame by using double brackets

census_joined2 = census_joined2[["Census Tract", "GEOID", "GEOID1", "Population", "Total households", "Household population", 
                       "Median household income",
                       "Per capita income", "Poverty count", "Pop. <18 years", "Pop. working age", "Pop. 65+ years",
                       "# employed, age 16+", "Unemployment count", "# persons 12th grade, no diploma",
                       "# persons age 25+ graduated high school", "# persons Associate's degree",
                       "# persons age 25+ with Bachelor's degree",
                       "Median age","Pop. white", "Pop. Black", "Pop. two or more races", "Pop. Hispanic origin", 
                       "Pop. Asian","Total pop. in occupied housing units by tenure", "Total occupied units",
                       "Total owner-occupied units", 
                       "Total renter-occupied units", "No_vehicle_available_owneroccupied_unit",
                       "No_vehicle_available_renteroccupied_unit",
                       "# Foreign-born", "# households with computer, no internet subscription",
                       "# households no computer",                                       
                       "Construction", "Manufacturing", "Retail Trade","Transportation, warehousing, utilities",                
                       "Finance, insurance, real estate", "Professional, scientific, mgmt, administrative",               
                       "Educational services, health care, social assistance", "Accommodations, food services, arts, entertainment",
                       "Public administration"      
                        ]]

census_joined2.head()

KeyError: "['# employed, age 16+', 'Unemployment count'] not in index"

In [ ]:
# Remove extra GEOID/ tract columns 
census_joined2 = census_joined2.drop(["GEOID1", "Census Tract"], axis=1)
census_joined2.head()

In [ ]:
# Save the ArcGIS acreage/ square mile data compiled by Patrick (csv file) in variable

data_GEOID_SQMI = "raw_data/BaltimoreCensusTract_Area.csv"

In [ ]:
# Create dataframes by reading the variable data from the code block above
df_GEOID_SQMI = pd.read_csv(data_GEOID_SQMI)
df_GEOID_SQMI

In [ ]:
# Need to rename the "GEOID " column as it has a space in the name so won't be able to merge on it (because other "GEOID" columns do not contain spaces)
# Replace the census variable codes (such as "B19013_001E") in the dataframe with text so it's understandable
df_GEOID_SQMI = df_GEOID_SQMI.rename(columns={"GEOID ": "GEOID"
                                    })

In [ ]:
df_GEOID_SQMI.dtypes

In [ ]:
# Change data type of the "GEOID" column so that it's a string/ text and not an integer
df_GEOID_SQMI["GEOID"] = df_GEOID_SQMI["GEOID"].astype('str')
df_GEOID_SQMI.dtypes

In [ ]:
# drop extraneous columns
GEOID_SQMI_df = df_GEOID_SQMI.drop(['NAMELSAD', 'Shape_Area'], axis=1)
GEOID_SQMI_df

In [ ]:
SQMI_df = GEOID_SQMI_df.sort_values('GEOID')
SQMI_df

In [ ]:
# combine GEOID_SQMI_df with census_joined2 df
census_temp = pd.merge(
    census_joined2, GEOID_SQMI_df, on="GEOID")
census_temp

In [ ]:
census_temp2 = census_temp.sort_values('GEOID')
census_temp2

In [ ]:
# GIS Analyst, Patrick, provided a csv (created from ARcGIS) that contains a key to match Baltimore's commercial corridors 
# with specific GEOIDS 

# Store filepath in a variable
corridor_key = "./CSVs/corr_key.csv"

# Read the file with the pandas library
corr_key_df2 = pd.read_csv(corridor_key)
corr_key_df2.dtypes

In [ ]:
# Change data type of the "GEOID" column so that it's a string/ text and not an integer
corr_key_df2["GEOID"] = corr_key_df2["GEOID"].astype('str')
corr_key_df2.dtypes

In [ ]:
corr_key_df2.columns

In [ ]:
corr_key_df2

In [ ]:
# see the number of rows (200) and columns (2) in the dataframe - so can compare it with the number of rows and columns in the census_joined df
corr_key_df2.shape

In [ ]:
corr_key_df = corr_key_df2.sort_values("GEOID")
corr_key_df

In [ ]:
# print all of the values in the "GEOID" column
#geoid_list = census_joined2['GEOID'].tolist()
#print(geoid_list)

In [ ]:
# merge the census_temp2 dataframe with the corr_key_df dataframe on the common column "GEOID"

corridors_df = pd.merge(
    census_temp2, corr_key_df, on="GEOID")

# remove any columns with NaN ("Not a Number") - used for missing values, by using .dropna()
#corridors_df = corridors_df.dropna()
corridors_df

In [ ]:
# Use .groupby and .agg to sum the amounts by corridor
corridors_sum = corridors_df.groupby(['Corridor'], as_index=False).agg(
    {"Population": sum,
     "Total households": sum,
     "Household population": sum,
     "Poverty count": sum,
     "Unemployment count": sum,
     "# employed, age 16+": sum,
     "# persons 12th grade, no diploma": sum,
     "# persons age 25+ graduated high school": sum,
     "# persons Associate's degree": sum,
     "# persons age 25+ with Bachelor's degree": sum,
     "# persons Associate's degree": sum,
     "Pop. Asian": sum,
     "Pop. Black": sum,
     "Pop. Hispanic origin": sum,
     "Pop. white": sum,
     "Pop. two or more races": sum,
     "# Foreign-born":sum,
     "Population_16_yrs_and_over": sum,
     "In_labor_force": sum,
     "Total pop. in occupied housing units by tenure":sum,
     "Total occupied units": sum,
     "Total owner-occupied units": sum,
     "Total renter-occupied units": sum,
     "No_vehicle_available_owneroccupied_unit": sum,
     "No_vehicle_available_renteroccupied_unit": sum,
     "# households with computer, no internet subscription":sum,
     "# households no computer":sum,
     "Pop. <18 years": sum,
     "Pop. working age": sum,
     "Pop. 65+ years": sum,
     "Commute_to_work_public_transportation": sum,
     "Construction": sum,               
     "Manufacturing": sum,
     "Retail Trade": sum,                
     "Transportation, warehousing, utilities": sum,                
     "Finance, insurance, real estate": sum,  
     "Professional, scientific, mgmt, administrative": sum,               
     "Educational services, health care, social assistance": sum,
     "Accommodations, food services, arts, entertainment": sum,
     "Public administration": sum, 
     "Area_Acres": sum,
     "Area_mi2":sum
    })
    
corridors_sum

In [ ]:
corridors_sum["Home ownership rate"] = 100 * \
    corridors_sum["Total owner-occupied units"].astype(
        int) / corridors_sum["Total occupied units"].astype(
        int) 

corridors_sum.head()

In [ ]:
# Round the home ownership rate to one decimal point; using "float" instead of "int" because want to use decimal points
corridors_sum["Home ownership rate"] = corridors_sum["Home ownership rate"].astype(float).round(1)


In [ ]:
# Add a new column for poverty rate (Poverty Count / Population)
corridors_sum["Poverty rate"] = 100 * \
    corridors_sum["Poverty count"].astype(
        int) / corridors_sum["Population"].astype(int)

corridors_sum["Poverty rate"] = corridors_sum["Poverty rate"].astype(float).round(1)


# Add a new column for unemployment rate (Employment Count / Population)
corridors_sum["Unemployment rate"] = 100 * \
    corridors_sum["Unemployment count"].astype(
        int) / corridors_sum["Population"].astype(int)

corridors_sum["Unemployment rate"] = corridors_sum["Unemployment rate"].astype(float).round(1)

corridors_sum.head()

In [ ]:
# Add a new column for average household size (Household population / Total households); see https://www.census.gov/quickfacts/fact/note/US/HSD410221#:~:text=Persons%20per%20household%2C%20or%20average,by%20the%20number%20of%20households.
corridors_sum["Average hh size"] = corridors_sum["Household population"].astype(
        int) / corridors_sum["Total households"].astype(int)

corridors_sum["Average hh size"] = corridors_sum["Average hh size"].astype(float).round(2)

corridors_sum.head()

In [ ]:
corridors_sum.columns

In [ ]:
# Add a new column for # of occupied units with no vehicle available (sum column values of owner-occupied units with no vehicles available
# and renter-occupied units with no vehicles available)

corridors_sum["Number units with no vehicle available"] = corridors_sum.apply(lambda row: row.No_vehicle_available_owneroccupied_unit + row.No_vehicle_available_renteroccupied_unit, axis=1) 
corridors_sum.head()


In [ ]:
# Add a new column for % of occupied units with no vehicle available 
# (owner-occupied units with no vehicle available + renter-occupied units with no vehicle available / Total occupied units)
corridors_sum["Percent units with no vehicle available"] = 100 * \
    corridors_sum["Number units with no vehicle available"].astype(
        int) / corridors_sum["Total occupied units"].astype(
        int) 

corridors_sum["Percent units with no vehicle available"] = corridors_sum["Percent units with no vehicle available"].astype(float).round(1)
corridors_sum

In [ ]:
corridors_sum["Percent commute to work public transportation"] = 100 * \
    corridors_sum["Commute_to_work_public_transportation"].astype(
        int) / corridors_sum["Workers_16_yrs_and_over"].astype(
        int) 

corridors_sum["Percent commute to work public transportation"] = corridors_sum["Percent commute to work public transportation"].astype(float).round(1)


In [ ]:
corridors_sum["Percent Asian"] = 100 * \
    corridors_sum["Pop. Asian"].astype(
        int) / corridors_sum["Population"].astype(
        int) 

corridors_sum["Percent Asian"] = corridors_sum["Percent Asian"].astype(float).round(1)

In [ ]:
corridors_sum["Percent Asian"] = 100 * \
    corridors_sum["Pop. Asian"].astype(
        int) / corridors_sum["Population"].astype(
        int) 

corridors_sum["Percent Asian"] = corridors_sum["Percent Asian"].astype(float).round(1)

In [ ]:
corridors_sum["Percent Black"] = 100 * \
    corridors_sum["Pop. Black"].astype(
        int) / corridors_sum["Population"].astype(
        int) 

corridors_sum["Percent Black"] = corridors_sum["Percent Black"].astype(float).round(1)

In [ ]:
corridors_sum["Percent Hispanic origin"] = 100 * \
    corridors_sum["Pop. Hispanic origin"].astype(
        int) / corridors_sum["Population"].astype(
        int) 

corridors_sum["Percent Hispanic origin"] = corridors_sum["Percent Hispanic origin"].astype(float).round(1)

In [ ]:
corridors_sum["Percent White"] = 100 * \
    corridors_sum["Pop. white"].astype(
        int) / corridors_sum["Population"].astype(
        int) 
corridors_sum["Percent White"] = corridors_sum["Percent White"].astype(float).round(1)


In [ ]:
corridors_sum["Percent two or more races"] = 100 * \
    corridors_sum["Pop. two or more races"].astype(
        int) / corridors_sum["Population"].astype(
        int) 
corridors_sum["Percent two or more races"] = corridors_sum["Percent two or more races"].astype(float).round(1)


In [ ]:
corridors_sum["Percent 12th grade, no diploma"] = 100 * \
    corridors_sum["# persons 12th grade, no diploma"].astype(
        int) / corridors_sum["Population"].astype(
        int) 
corridors_sum["Percent 12th grade, no diploma"] = corridors_sum["Percent 12th grade, no diploma"].astype(float).round(1)


In [ ]:
corridors_sum["Percent graduated high school"] = 100 * \
    corridors_sum["# persons age 25+ graduated high school"].astype(
        int) / corridors_sum["Population"].astype(
        int) 
corridors_sum["Percent graduated high school"] = corridors_sum["Percent graduated high school"].astype(float).round(1)


In [ ]:
corridors_sum["Percent earned Associate's degree"] = 100 * \
    corridors_sum["# persons Associate's degree"].astype(
        int) / corridors_sum["Population"].astype(
        int) 
corridors_sum["Percent earned Associate's degree"] = corridors_sum["Percent earned Associate's degree"].astype(float).round(1)


In [ ]:
corridors_sum["Percent earned Bachelor's degree"] = 100 * \
    corridors_sum["# persons age 25+ with Bachelor's degree"].astype(
        int) / corridors_sum["Population"].astype(
        int) 
corridors_sum["Percent earned Bachelor's degree"] = corridors_sum["Percent earned Bachelor's degree"].astype(float).round(1)


In [ ]:
corridors_sum["Percent computer, no internet subscription"] = 100 * \
    corridors_sum["# households with computer, no internet subscription"].astype(
        int) / corridors_sum["Population"].astype(
        int) 
corridors_sum["Percent computer, no internet subscription"] = corridors_sum["Percent computer, no internet subscription"].astype(float).round(1)


In [ ]:
corridors_sum["Percent no computer"] = 100 * \
    corridors_sum["# households no computer"].astype(
        int) / corridors_sum["Population"].astype(
        int) 
corridors_sum["Percent no computer"] = corridors_sum["Percent no computer"].astype(float).round(1)


In [ ]:
corridors_sum["Percent less than 18 yrs old"] = 100 * \
    corridors_sum["Pop. <18 years"].astype(
        int) / corridors_sum["Population"].astype(
        int) 
corridors_sum["Percent less than 18 yrs old"] = corridors_sum["Percent less than 18 yrs old"].astype(float).round(1)


In [ ]:
# working age is 18-64 yrs old
corridors_sum["Percent working age"] = 100 * \
    corridors_sum["Pop. working age"].astype(
        int) / corridors_sum["Population"].astype(
        int) 
corridors_sum["Percent working age"] = corridors_sum["Percent working age"].astype(float).round(1)


In [ ]:
corridors_sum["Percent 65 or older"] = 100 * \
    corridors_sum["Pop. 65+ years"].astype(
        int) / corridors_sum["Population"].astype(
        int) 
corridors_sum["Percent 65 or older"] = corridors_sum["Percent 65 or older"].astype(float).round(1)


In [ ]:
corridors_sum["Percent 65 or older"] = 100 * \
    corridors_sum["Pop. 65+ years"].astype(
        int) / corridors_sum["Population"].astype(
        int) 
corridors_sum["Percent 65 or older"] = corridors_sum["Percent 65 or older"].astype(float).round(1)


In [ ]:
corridors_sum["Percent Foreign-born"] = 100 * \
    corridors_sum["# Foreign-born"].astype(
        int) / corridors_sum["Population"].astype(
        int) 
corridors_sum["Percent Foreign-born"] = corridors_sum["Percent Foreign-born"].astype(float).round(1)
corridors_sum

In [ ]:
corridors_sum["Percent Manufacturing"] = 100 * \
    corridors_sum["Manufacturing"].astype(
        int) / corridors_sum["Population"].astype(
        int) 
corridors_sum["Percent Manufacturing"] = corridors_sum["Percent Manufacturing"].astype(float).round(1)


In [ ]:
corridors_sum["Percent Retail Trade"] = 100 * \
    corridors_sum["Retail Trade"].astype(
        int) / corridors_sum["Population"].astype(
        int) 
corridors_sum["Percent Retail Trade"] = corridors_sum["Percent Retail Trade"].astype(float).round(1)


In [ ]:
corridors_sum["Percent Transportation, warehousing, utilities"] = 100 * \
    corridors_sum["Transportation, warehousing, utilities"].astype(
        int) / corridors_sum["Population"].astype(
        int) 
corridors_sum["Percent Transportation, warehousing, utilities"] = corridors_sum["Percent Transportation, warehousing, utilities"].astype(float).round(1)


In [ ]:
corridors_sum["Percent Finance, insurance, real estate"] = 100 * \
    corridors_sum["Finance, insurance, real estate"].astype(
        int) / corridors_sum["Population"].astype(
        int) 
corridors_sum["Percent Finance, insurance, real estate"] = corridors_sum["Percent Finance, insurance, real estate"].astype(float).round(1)


In [ ]:
corridors_sum["Percent Professional, scientific, mgmt, administrative"] = 100 * \
    corridors_sum["Professional, scientific, mgmt, administrative"].astype(
        int) / corridors_sum["Population"].astype(
        int) 
corridors_sum["Percent Professional, scientific, mgmt, administrative"] = corridors_sum["Percent Professional, scientific, mgmt, administrative"].astype(float).round(1)


In [ ]:
corridors_sum["Percent Educational services, health care, social assistance"] = 100 * \
    corridors_sum["Educational services, health care, social assistance"].astype(
        int) / corridors_sum["Population"].astype(
        int) 
corridors_sum["Percent Educational services, health care, social assistance"] = corridors_sum["Percent Educational services, health care, social assistance"].astype(float).round(1)


In [ ]:
corridors_sum["Percent Accommodations, food services, arts, entertainment"] = 100 * \
    corridors_sum["Accommodations, food services, arts, entertainment"].astype(
        int) / corridors_sum["Population"].astype(
        int) 
corridors_sum["Percent Accommodations, food services, arts, entertainment"] = corridors_sum["Percent Accommodations, food services, arts, entertainment"].astype(float).round(1)


In [ ]:
corridors_sum["Percent Public administration"] = 100 * \
    corridors_sum["Public administration"].astype(
        int) / corridors_sum["Population"].astype(
        int) 
corridors_sum["Percent Public administration"] = corridors_sum["Percent Public administration"].astype(float).round(1)
corridors_sum

In [ ]:
# Export file as a CSV, without the Pandas index, but with the header
corridors_sum.to_csv("CSVs/Corridors_only_2021.csv", index = False, header=True)

In [ ]:
corridors_sum.columns

In [ ]:
corridors_2021_final = corridors_sum.copy()
corridors_2021_final

In [ ]:
# Export df as an Excel file
corridors_2021_final.to_excel("CommCorr_demographics_2021.xlsx", index = False)

In [ ]:
# import csv file of Baltimore City (as a whole) demographics
# first save the data from the csv in variable, "baltcity_2021" 

baltcity_2021 = "BaltimoreCity/BaltCity_demographics_2021.csv"

In [ ]:
# Create dataframes by reading the variable data from the code block above
balt_df2 = pd.read_csv(baltcity_2021)
balt_df2

In [ ]:
balt_df = balt_df2.rename(columns={"Area_Acres.1": "Area_Acres",
                                   "Area_mi2.1": "Area_mi2"
                                  })
balt_df

In [ ]:
# Merge the 2 dataframes together using pd.concat
frames = [balt_df, corridors_2021_final]
corridors_balt= pd.concat(frames)
corridors_balt

In [ ]:
# Drop columns that we don't want in the table at this time
corridors_balt = corridors_balt.drop(["Median household income", "Median age", "Per capita income"], axis=1)
corridors_balt

In [ ]:
#corridors_balt.to_excel("CommCorr_Tableau-2021/corr_balt_2021.xlsx")

In [ ]:
# Export file as a CSV, without the Pandas index, but with the header
# Do not run this last code block until you have all previous code blocks in their final form:

#corridors_sum.to_csv("CSVs/CommCorr_demographics_2021.csv", index = False, header=True)

In [ ]:
# import csv file of Maryland  demographics
# first save the data from the csv in variable, "MD_2021" 

MD_2021 = "MD_demographics_2021.csv"

In [ ]:
# Create dataframes by reading the variable data from the code block above
MD_df = pd.read_csv(MD_2021)
MD_df

In [ ]:
# Merge 2 dataframes together using pd.concat
frames2 = [MD_df, corridors_balt]
corridors_balt_MD = pd.concat(frames2)
corridors_balt_MD

In [ ]:
# Calculate population density  see: https://www.census.gov/quickfacts/fact/note/US/LND110210
# Density is expressed as "population per square mile(kilometer)"
# Divide total population (or # of housing units)/ by land area of the entity measured in square miles
corridors_sum['Pop. density per Sq Mile'] = corridors_sum['Population'].astype(
                                    float) / corridors_sum['Area_mi2'].astype(
                                    float)
corridors_sum   

In [ ]:
# Round the pop. density rate to one decimal point; using "float" instead of "int" because want to use decimal points
corridors_sum["Pop. density per Sq Mile"] = corridors_sum["Pop. density per Sq Mile"].astype(float).round(1)
corridors_sum

In [ ]:
#### FILE TITLE: see Sharepoint/ Commercial Corridors 

# Export file as an Excel file
#corridors_balt_MD.to_excel("MD_Balt_corridor_demogr_2021.xlsx", index = False)

In [ ]:
# import csv file created from Commercial_corridors/vacant_comm_bldgs Jupyter nb
# save the data from the csv in variable, "vacants_2021" 

#vacants_2021 = "vacants_USPS_2021Q4.csv"

In [ ]:
# Create dataframe ("vacants_USPS_2021") by reading the variable data from the code block above
#vacants_USPS_2021 = pd.read_csv(vacants_2021)
#vacants_USPS_2021

In [ ]:
#census_2021_formatted = census_2021
#census_2021_formatted.head()

In [ ]:
"""
# Use .map to format columns (helpful resource for this: https://towardsdatascience.com/apply-thousand-separator-and-other-formatting-to-pandas-dataframe-45f2f4c7ab01)
# Note: once you format values in a column, they are changed to strings (see cell below to see data types of each column)
# I will use the census_2017_FINAL dataframe to use for analysis as needed (can do calculations with number data types but not strings)
# You may need to restart the kernel after you format
census_2020_formatted["Median household income"] = census_2020_FINAL["Median household income"].map("${:.2f}".format)
census_2020_formatted["Per capita income"] = census_2020_FINAL["Per capita income"].map("${:.2f}".format)
census_2020_formatted["Population"] = census_2020_formatted["Population"].map("{:,.0f}".format)
census_2020_formatted["Poverty count"] = census_2020_formatted["Poverty count"].map("{:,.0f}".format)
census_2020_formatted["Poverty rate"] = census_2020_formatted["Poverty rate"].map("{:.2%}".format)
census_2020_formatted["Unemployment rate"] = census_2020_formatted["Unemployment rate"].map("{:.2%}".format)

census_2020_formatted = census_2020_formatted.reset_index(drop=True)
census_2020_formatted.head()
"""